### Pipeline

The entire pipeline for comparing and see the correlation between the UST-HRV and the Standard one.

This process consist of several steps:
- Data Extraction 
- rPPG Validation (MAE /RMSE) for each Scenario with bland-altman aggrement 
- Making a section for each scenario (30 sec, 1min and standard one 3 min) for UBFC-Phys, (30 sec and 1 min) for Physio Itera
- Plot the correlation and check the limit within bland-altman
- Done.

### Kesimpulan

- Overall the Facial Landmark unggul / better than Face detector
- For rest task, it seems the `Face Detector` / `Facial Landmark` shows similar result with lower MAE / RMSE and low difference bias in Bland-Altman plot.
- Green is the worst one for both.
- Task 3 is the Face Detector having advantages, it's having slightly lower MAE / RMSE score compare to the Landmark, but still I don't consider it valid since the data maybe have some artifcats.

- For Physio-Itera
    - For some reason the Face Detector and Facial Landmark have similar result for in MAE and from the mean bias bland-altman shows higher mean bias.

- For future test, let's use the Base Task 1 and Task 2

## Data Extraction

This will structure the folder so it can be easier to access, all rPPG methods already be in the .npy files for easier acccess.

In [1]:
## Import Dependencies
import numpy as np
import mediapipe as mp
import pandas as pd
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
import scipy
import os
import neurokit2 as nk
from prettytable import PrettyTable

## UBFC-Phys Scenario

Plot the data and see the data variation between models within tables views and compare with the GT

In [ ]:
import scipy.signal


root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44", "s45", "s46", "s47", "s48", "s49", "s50", "s51","s52","s53","s54","s55","s56"]
tasks = ["T3"]

table = PrettyTable()

## Create a table for the HRV Metrics for GT and RPPG
table.field_names = [
   "Subject",
   "Elgandi_PR_RPPG",
   "Elgandi_PR_GT",
   "Bishop_PR_RPPG",
    "Bishop_PR_GT",
    "Find_Peaks_PR_RPPG",
    "Find_Peaks_PR_GT",
    "Elgandi_MeanNN_RPPG",
    "Elgandi_MeanNN_GT",
    "Bishop_MeanNN_RPPG",
    "Bishop_MeanNN_GT",
    "Find_Peaks_MeanNN_RPPG",
    "Find_Peaks_MeanNN_GT",
    "Elgandi_SDNN_RPPG",
    "Elgandi_SDNN_GT",
    "Bishop_SDNN_RPPG",
    "Bishop_SDNN_GT",
    "Find_Peaks_SDNN_RPPG",
    "Find_Peaks_SDNN_GT",
    "Elgandi_RMSSD_RPPG",
    "Elgandi_RMSSD_GT",
    "Bishop_RMSSD_RPPG",
    "Bishop_RMSSD_GT",
    "Find_Peaks_RMSSD_RPPG",
    "Find_Peaks_RMSSD_GT",
    "Elgandi_pNN50_RPPG",
    "Elgandi_pNN50_GT",
    "Bishop_pNN50_RPPG",
    "Bishop_pNN50_GT",
    "Find_Peaks_pNN50_RPPG",
    "Find_Peaks_pNN50_GT",
    "Elgandi_LF_RPPG",
    "Elgandi_LF_GT",
    "Bishop_LF_RPPG",
    "Bishop_LF_GT",
    "Find_Peaks_LF_RPPG",
    "Find_Peaks_LF_GT",
    "Elgandi_HF_RPPG",
    "Elgandi_HF_GT",
    "Bishop_HF_RPPG",
    "Bishop_HF_GT",
    "Find_Peaks_HF_RPPG",
    "Find_Peaks_HF_GT",
    "Elgandi_LF_HF_RPPG",
    "Elgandi_LF_HF_GT",
    "Bishop_LF_HF_RPPG",
    "Bishop_LF_HF_GT",
    "Find_Peaks_LF_HF_RPPG",
    "Find_Peaks_LF_HF_GT"   
]

for subject in subjects:
    if not os.path.exists(f"{root_path}/{subject}"):
        print(f"Subject {subject} not found, skipping")
    
    for task in tasks:
        print(f"Processing subject {subject} for task {task}")

        # Create a dictionary to store the HRV metrics for RPPG and GT for each methods of peak detection
    
        rppg_hrv_metrics_elgendi = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        gt_hrv_metrics_elgendi = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        rppg_hrv_metrics_bishop = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        gt_hrv_metrics_bishop = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        rppg_hrv_metrics_find_peaks = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        gt_hrv_metrics_find_peaks = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        # Peaks based on the Elgendi method Peak detection

        # Example : POS methods
        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))

        # Open the GT file
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        
        # Flatten the value for the GT
        GT = GT.flatten()

        # Process all the signals
        pos_hr = nk.ppg_process(pos, sampling_rate=35)[0]["PPG_Rate"]
        gt_hr = nk.ppg_process(GT, sampling_rate=64)[0]["PPG_Rate"]

        # Peaks based on the Elgandi method Peak detection

        fs = 35
        signals, _ = nk.ppg_process(pos, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs, correct_artifacts=True)

        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        rppg_hrv_metrics_elgendi['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        rppg_hrv_metrics_elgendi['SDNN'].append(hrv_time['HRV_SDNN'].item())
        rppg_hrv_metrics_elgendi['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        rppg_hrv_metrics_elgendi['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        rppg_hrv_metrics_elgendi['LF'].append(hrv_freq['HRV_LF'].item())
        rppg_hrv_metrics_elgendi['HF'].append(hrv_freq['HRV_HF'].item())
        rppg_hrv_metrics_elgendi['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        rppg_hrv_metrics_elgendi['PR'].append(signals['PPG_Rate'][0].item())

        fs = 64
        signals, _ = nk.ppg_process(GT, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs, correct_artifacts=True)

        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        gt_hrv_metrics_elgendi['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        gt_hrv_metrics_elgendi['SDNN'].append(hrv_time['HRV_SDNN'].item())
        gt_hrv_metrics_elgendi['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        gt_hrv_metrics_elgendi['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        gt_hrv_metrics_elgendi['LF'].append(hrv_freq['HRV_LF'].item())
        gt_hrv_metrics_elgendi['HF'].append(hrv_freq['HRV_HF'].item())
        gt_hrv_metrics_elgendi['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        gt_hrv_metrics_elgendi['PR'].append(signals["PPG_Rate"][0].item())

        # ------------------------------------------------
        # Peaks based on the Bishop method Peak detection

        # Process all the signals
        pos_hr = nk.ppg_process(pos, sampling_rate=35)[0]["PPG_Rate"]
        gt_hr = nk.ppg_process(GT, sampling_rate=64)[0]["PPG_Rate"]

        # Peaks based on the Elgandi method Peak detection

        fs = 35
        signals, _ = nk.ppg_process(pos, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs, method="bishop", correct_artifacts=True)

        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        rppg_hrv_metrics_bishop['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        rppg_hrv_metrics_bishop['SDNN'].append(hrv_time['HRV_SDNN'].item())
        rppg_hrv_metrics_bishop['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        rppg_hrv_metrics_bishop['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        rppg_hrv_metrics_bishop['LF'].append(hrv_freq['HRV_LF'].item())
        rppg_hrv_metrics_bishop['HF'].append(hrv_freq['HRV_HF'].item())
        rppg_hrv_metrics_bishop['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        rppg_hrv_metrics_bishop['PR'].append(signals['PPG_Rate'][0].item())

        fs = 64
        signals, _ = nk.ppg_process(GT, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs, method="bishop", correct_artifacts=True)

        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        gt_hrv_metrics_bishop['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        gt_hrv_metrics_bishop['SDNN'].append(hrv_time['HRV_SDNN'].item())
        gt_hrv_metrics_bishop['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        gt_hrv_metrics_bishop['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        gt_hrv_metrics_bishop['LF'].append(hrv_freq['HRV_LF'].item())
        gt_hrv_metrics_bishop['HF'].append(hrv_freq['HRV_HF'].item())
        gt_hrv_metrics_bishop['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        gt_hrv_metrics_bishop['PR'].append(signals["PPG_Rate"][0].item())

        # ------------------------------------------------
        # Peaks based on the Find Peaks method Peak detection        

        ## Getting the HR
        pos_hr = nk.ppg_process(pos, sampling_rate=35)[0]["PPG_Rate"]
        gt_hr = nk.ppg_process(GT, sampling_rate=64)[0]["PPG_Rate"]

        # Peaks based on the scipy find_peaks method Peak detection
        fs = 35
        signals, _ = nk.ppg_process(pos, sampling_rate=fs)
        peaks, _ = scipy.signal.find_peaks(signals['PPG_Clean'], prominence=0.2, distance=fs*0.4)
        
        # Compute the pulse rate based on the scipy find_peaks method
        pos_hr = np.mean(60 / (np.diff(peaks) / fs))   # Convert to beats per minute
        rppg_hrv_metrics_find_peaks['PR'].append(pos_hr)

        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        rppg_hrv_metrics_find_peaks['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        rppg_hrv_metrics_find_peaks['SDNN'].append(hrv_time['HRV_SDNN'].item())
        rppg_hrv_metrics_find_peaks['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        rppg_hrv_metrics_find_peaks['pNN50'].append(hrv_time['HRV_pNN50'].item())
        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        rppg_hrv_metrics_find_peaks['LF'].append(hrv_freq['HRV_LF'].item())
        rppg_hrv_metrics_find_peaks['HF'].append(hrv_freq['HRV_HF'].item())
        rppg_hrv_metrics_find_peaks['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        fs = 64
        signals, _ = nk.ppg_process(GT, sampling_rate=fs)
        peaks, _ = scipy.signal.find_peaks(signals['PPG_Clean'], prominence=0.2, distance=fs*0.4)
        # Compute the pulse rate based on the scipy find_peaks method
        gt_hr = np.mean(60 / (np.diff(peaks) / fs))
        gt_hrv_metrics_find_peaks['PR'].append(gt_hr)
        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        gt_hrv_metrics_find_peaks['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        gt_hrv_metrics_find_peaks['SDNN'].append(hrv_time['HRV_SDNN'].item())
        gt_hrv_metrics_find_peaks['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        gt_hrv_metrics_find_peaks['pNN50'].append(hrv_time['HRV_pNN50'].item())
        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        gt_hrv_metrics_find_peaks['LF'].append(hrv_freq['HRV_LF'].item())
        gt_hrv_metrics_find_peaks['HF'].append(hrv_freq['HRV_HF'].item())
        gt_hrv_metrics_find_peaks['LF_HF'].append(hrv_freq['HRV_LFHF'].item())
        
        ## Add the metrics to the table
        table.add_row([
            subject,
            rppg_hrv_metrics_elgendi['PR'][0],
            gt_hrv_metrics_elgendi['PR'][0],
            rppg_hrv_metrics_bishop['PR'][0],
            gt_hrv_metrics_bishop['PR'][0],
            rppg_hrv_metrics_find_peaks['PR'][0],
            gt_hrv_metrics_find_peaks['PR'][0],
            rppg_hrv_metrics_elgendi['MeanNN'][0],
            gt_hrv_metrics_elgendi['MeanNN'][0],
            rppg_hrv_metrics_bishop['MeanNN'][0],
            gt_hrv_metrics_bishop['MeanNN'][0],
            rppg_hrv_metrics_find_peaks['MeanNN'][0],
            gt_hrv_metrics_find_peaks['MeanNN'][0],
            rppg_hrv_metrics_elgendi['SDNN'][0],
            gt_hrv_metrics_elgendi['SDNN'][0],
            rppg_hrv_metrics_bishop['SDNN'][0],
            gt_hrv_metrics_bishop['SDNN'][0],
            rppg_hrv_metrics_find_peaks['SDNN'][0],
            gt_hrv_metrics_find_peaks['SDNN'][0],
            rppg_hrv_metrics_elgendi['RMSSD'][0],
            gt_hrv_metrics_elgendi['RMSSD'][0],
            rppg_hrv_metrics_bishop['RMSSD'][0],
            gt_hrv_metrics_bishop['RMSSD'][0],
            rppg_hrv_metrics_find_peaks['RMSSD'][0],
            gt_hrv_metrics_find_peaks['RMSSD'][0],
            rppg_hrv_metrics_elgendi['pNN50'][0],
            gt_hrv_metrics_elgendi['pNN50'][0],
            rppg_hrv_metrics_bishop['pNN50'][0],
            gt_hrv_metrics_bishop['pNN50'][0],
            rppg_hrv_metrics_find_peaks['pNN50'][0],
            gt_hrv_metrics_find_peaks['pNN50'][0],
            rppg_hrv_metrics_elgendi['LF'][0],
            gt_hrv_metrics_elgendi['LF'][0],
            rppg_hrv_metrics_bishop['LF'][0],
            gt_hrv_metrics_bishop['LF'][0],
            rppg_hrv_metrics_find_peaks['LF'][0],
            gt_hrv_metrics_find_peaks['LF'][0],
            rppg_hrv_metrics_elgendi['HF'][0],
            gt_hrv_metrics_elgendi['HF'][0],
            rppg_hrv_metrics_bishop['HF'][0],
            gt_hrv_metrics_bishop['HF'][0],
            rppg_hrv_metrics_find_peaks['HF'][0],
            gt_hrv_metrics_find_peaks['HF'][0],
            rppg_hrv_metrics_elgendi['LF_HF'][0],
            gt_hrv_metrics_elgendi['LF_HF'][0],
            rppg_hrv_metrics_bishop['LF_HF'][0],
            gt_hrv_metrics_bishop['LF_HF'][0],  
            rppg_hrv_metrics_find_peaks['LF_HF'][0],
            gt_hrv_metrics_find_peaks['LF_HF'][0]
        ])

# Print the table
print(table)

## Create a dataFrame from the table
df = pd.DataFrame(table.rows, columns=table.field_names)
# Save the DataFrame to a CSV file of the HRV Metrics
df.to_csv("Stress-HRV_Metrics_UBFC-Phys.csv", index=False)



Processing subject s41 for task T1
Processing subject s42 for task T1
Processing subject s43 for task T1
Processing subject s44 for task T1
Processing subject s45 for task T1
Processing subject s46 for task T1
Processing subject s47 for task T1
Processing subject s48 for task T1
Processing subject s49 for task T1
Processing subject s50 for task T1
Processing subject s51 for task T1
Processing subject s52 for task T1
Processing subject s53 for task T1
Processing subject s54 for task T1
Processing subject s55 for task T1
Processing subject s56 for task T1
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+--------------------+--------------------+------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+

In [3]:
## Compute the MAE / RMSE for the RPPG and GT signals
root_path = "UBFC-Phys"
subjects = ["s41", "s42", "s43", "s44","s45","s46","s47","s48","s49","s50","s51","s52", "s53","s54","s55","s56",]
tasks = ["T1"]

table = PrettyTable()
## Create a table for the MAE / RMSE HRV metrics for GT and RPPG
table.field_names = ["Subject", "MAE_PR", "RMSE_PR", "MAE_MeanNN", "RMSE_MeanNN",
                     "MAE_SDNN", "RMSE_SDNN", "MAE_RMSSD", "RMSE_RMSSD", 
                     "MAE_pNN50", "RMSE_pNN50", "MAE_LF", "RMSE_LF", 
                     "MAE_HF", "RMSE_HF", "MAE_LF_HF_Ratio", "RMSE_LF_HF_Ratio"]

for subject in subjects:
    if not os.path.exists(f"{root_path}/{subject}"):
        print(f"Subject {subject} not found, skipping")
    
    for task in tasks:

        ## Create a dataFrame for storing the HRV Metrics
        rppg_hrv_metrics = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        gt_hrv_metrics = {
            'MeanNN': [],
            'SDNN': [],
            'RMSSD': [],
            'pNN50': [],
            'LF': [],
            'HF': [],
            'LF_HF': [],
            'PR' : [],
        }

        pos = np.load(os.path.join(root_path, subject, f"Landmark_{subject}_{task}_POS_rppg.npy"))
        # Open the GT file
        GT = pd.read_csv(os.path.join(root_path, subject, f"bvp_{subject}_{task}.csv")).values
        GT = GT.flatten()


        # Process all the signals
        pos_hr = nk.ppg_process(pos, sampling_rate=35)[0]["PPG_Rate"].mean()
        gt_hr = nk.ppg_process(GT, sampling_rate=64)[0]["PPG_Rate"].mean()

        fs = 35
        signals, _ = nk.ppg_process(pos, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs)

        ## Compute the HRV Metrics
        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        rppg_hrv_metrics['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        rppg_hrv_metrics['SDNN'].append(hrv_time['HRV_SDNN'].item())
        rppg_hrv_metrics['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        rppg_hrv_metrics['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        rppg_hrv_metrics['LF'].append(hrv_freq['HRV_LF'].item())
        rppg_hrv_metrics['HF'].append(hrv_freq['HRV_HF'].item())
        rppg_hrv_metrics['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        rppg_hrv_metrics['PR'].append(signals['PPG_Rate'][0].item())

        fs = 64
        signals, _ = nk.ppg_process(GT, sampling_rate=fs)
        peaks, _ = nk.ppg_peaks(signals['PPG_Clean'], sampling_rate=fs)

        hrv_time = nk.hrv_time(peaks, sampling_rate=fs)
        gt_hrv_metrics['MeanNN'].append(hrv_time['HRV_MeanNN'].item())
        gt_hrv_metrics['SDNN'].append(hrv_time['HRV_SDNN'].item())
        gt_hrv_metrics['RMSSD'].append(hrv_time['HRV_RMSSD'].item())
        gt_hrv_metrics['pNN50'].append(hrv_time['HRV_pNN50'].item())

        # Frequency domain metrics
        hrv_freq = nk.hrv_frequency(peaks, sampling_rate=fs)
        gt_hrv_metrics['LF'].append(hrv_freq['HRV_LF'].item())
        gt_hrv_metrics['HF'].append(hrv_freq['HRV_HF'].item())
        gt_hrv_metrics['LF_HF'].append(hrv_freq['HRV_LFHF'].item())

        gt_hrv_metrics['PR'].append(signals["PPG_Rate"][0].item())

        ## Compute the MAE / RMSE for the RPPG and GT signals
        rppg_pr = rppg_hrv_metrics['PR'][0]
        gt_pr = gt_hrv_metrics['PR'][0]
        rppg_meanNN = rppg_hrv_metrics['MeanNN'][0]
        gt_meanNN = gt_hrv_metrics['MeanNN'][0]
        rppg_sdnn = rppg_hrv_metrics['SDNN'][0]
        gt_sdnn = gt_hrv_metrics['SDNN'][0]
        rppg_rmssd = rppg_hrv_metrics['RMSSD'][0]
        gt_rmssd = gt_hrv_metrics['RMSSD'][0]
        rppg_pnn50 = rppg_hrv_metrics['pNN50'][0]
        gt_pnn50 = gt_hrv_metrics['pNN50'][0]
        rppg_lf = rppg_hrv_metrics['LF'][0]
        gt_lf = gt_hrv_metrics['LF'][0]
        rppg_hf = rppg_hrv_metrics['HF'][0]
        gt_hf = gt_hrv_metrics['HF'][0]
        rppg_lf_hf = rppg_hrv_metrics['LF_HF'][0]
        gt_lf_hf = gt_hrv_metrics['LF_HF'][0]

        # Compute the MAE
        mae_pr = np.abs(rppg_pr - gt_pr)
        mae_meanNN = np.abs(rppg_meanNN - gt_meanNN)
        mae_sdnn = np.abs(rppg_sdnn - gt_sdnn)
        mae_rmssd = np.abs(rppg_rmssd - gt_rmssd)
        mae_pnn50 = np.abs(rppg_pnn50 - gt_pnn50)
        mae_lf = np.abs(rppg_lf - gt_lf)
        mae_hf = np.abs(rppg_hf - gt_hf)
        mae_lf_hf = np.abs(rppg_lf_hf - gt_lf_hf)

        # Compute the RMSE
        rmse_pr = np.sqrt((rppg_pr - gt_pr) ** 2)
        rmse_meanNN = np.sqrt((rppg_meanNN - gt_meanNN) ** 2)
        rmse_sdnn = np.sqrt((rppg_sdnn - gt_sdnn) ** 2)
        rmse_rmssd = np.sqrt((rppg_rmssd - gt_rmssd) ** 2)
        rmse_pnn50 = np.sqrt((rppg_pnn50 - gt_pnn50) ** 2)
        rmse_lf = np.sqrt((rppg_lf - gt_lf) ** 2)
        rmse_hf = np.sqrt((rppg_hf - gt_hf) ** 2)
        rmse_lf_hf = np.sqrt((rppg_lf_hf - gt_lf_hf) ** 2)

        ## Add the metrics to the table
        table.add_row([subject, 
                       mae_pr, rmse_pr,
                       mae_meanNN, rmse_meanNN,
                       mae_sdnn, rmse_sdnn,
                       mae_rmssd, rmse_rmssd,
                       mae_pnn50, rmse_pnn50,
                       mae_lf, rmse_lf,
                       mae_hf, rmse_hf,
                       mae_lf_hf, rmse_lf_hf])

## Print the table
print(table)



+---------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+----------------------+----------------------+-----------------------+-----------------------+
| Subject |        MAE_PR       |       RMSE_PR       |     MAE_MeanNN     |    RMSE_MeanNN     |      MAE_SDNN      |     RMSE_SDNN      |     MAE_RMSSD      |     RMSE_RMSSD     |     MAE_pNN50      |     RMSE_pNN50     |         MAE_LF        |        RMSE_LF        |        MAE_HF        |       RMSE_HF        |    MAE_LF_HF_Ratio    |    RMSE_LF_HF_Ratio   |
+---------+---------------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------------+---------------------

---

## Notes: 

It seems for task 2 and 3 it will be resulted in that same pattern since the nature signal of the rPPG and PPG are completely differ, and rPPG is good enough to detect the HR beat, not with the HRV for detail regardin the human condition